In [1]:
from datetime import datetime
from pprint import pprint
import pandas as pd
from dataclasses import dataclass

import sys
sys.path.append('..')
import doctable

First we'll quickly review [Python dataclasses (introduced in Python 3.7)](https://realpython.com/python-data-classes/). Lets start with a simple python object called `MyDataOriginal`. Here we create a constructor with some default values. If we wanted to, we could create class overloads for `__repr__`, `__lt__`, `__gt__`, `__eq__`, etc. Convenient overloads for printing and comparing objects.

In [2]:
class MyDataOriginal:
    def __init__(self, name, height=None):
        self.name = name
        self.height = height
MyDataOriginal('a name')

With a dataclass, we can implement this and other convenient class overloads at once using the `@dataclass` decorator. The following class is equivalent to the previous. See here that it has overloaded `__init__`, `__repr__`, and `__eq__`, among others.

In [3]:
@dataclass
class MyDataClass:
    name: str
    height: float = None
MyDataClass('Devin'), MyDataClass('Devin') == MyDataClass('Cornell')

(MyDataClass(name='Devin', height=None), False)

We can turn this into a schema for doctable by making only some minor modifications. First, we assign default values for our member variables using the function `doctable.Col()`. These are not as special as they seem - they simply set the default value of the variable to be a doctable `EmptyValue`, for when the object is retreived from the database, but the given column was not selected.

In [4]:
@dataclass
class MyData(doctable.DocTableSchema):
    name: str = doctable.Col()
    height: float = doctable.Col()
MyData('deveroni')

MyData(name='deveroni', height=EmptyValue)

We use it as a schema by providing it as the schema argument to the doctable constructor. Here we create an in-memory table. Of course, we can see that this table has no rows.

In [5]:
db = doctable.DocTable(schema=MyData, target=':memory:')
db.head()

""


In [6]:
isinstance(db._schema, type)

True

To add a row to this table, simply insert a `MyData` instance. 

In [7]:
o = MyData('Tyrone', 4.5)
db.insert(o)
db.head()

,name,height
0,Tyrone,4.5
